## Battery characterization: electroanalytical techniques

### Standard constant current constant voltage cycling

### DCIR 

### EIS

Electrochemical impedance spectroscopy (EIS) is an electroanalytical method where a potential or current with a small amplitude perturbation is applied.

The frequency (ω) of this perturbation can be changed, to measure the resulting cell impedance (Z)

The cell impedance (Z) is related to anything that restricts current flow: electrical resistances (poor connections, passivating films), interfacial capacitances, mass transfer of reactants, and reaction kinetics. 

Batteries and other electrochemical cells can be treated, or modeled, as a combination of circuits (the electrolyte acts as a resistor, the electrode/electrolyte interface acts as a capacitor).

The simplest unit is a resistor. The impedance of a resistor is: 

Z = V/I = R  → This is a point on the Nyquist plot, which plots complex versus real impedance values at various values of frequency (ω). 

Zreal = constant, Zimag = 0


A capacitor is a vertical line on the y-axis. Zreal = 0, Zimag = 1/(2π*ω*Cd)

A resistor and capacitor in series is a vertical line away from the y-axis. 
Zreal = 5, Zimag = 1/(2π*ω*Cd)

A capacitor and resistor in parallel is a semicircle that starts at 0. This is now getting closer to the behavior of an electrochemical interface. 

The EIS curve depicted represents the Nyquist plot of a simple circuit that represents an electrochemical interface. Assuming the electrochemistry is simple, the equivalent circuit is comprised of a resistor in series with a parallel capacitor/resistor. The lone resistance can be attributed to an ohmic resistance, and the parallel capacitor/resistor an electrochemical interface that has capacitance and a charge transfer resistance.

A semicircle can be observed for a well behaved system, where the high frequency x-axis intercept is the ohmic resistance, and the diameter of the semicircle is the interfacial resistance. 

In real systems such as batteries, EIS data generally gives a less-than-ideal semicircle due to the presence of many different interfaces

In a real system, the low frequency regime corresponds to a state of mass transfer control. The charge transfer resistance can be estimated by modeling the semicircle. 

This Nyquist plot is of an ideal system. In real systems, the low frequency regime likely departs from the semicircle due to the influence of the Warburg impedance (diffusion processes). 

EIS curves can be fit using equivalent circuit models. 

For instance, one could assume a resistor in series with a resistor + capacitor, and another resistor + capacitor. 

However, it is important to avoid over-fitting. 


### GITT

An electrochemical analysis technique used to measure kinetics. The technique uses a potentiostat to collect both transient (i.e. immediate iR drop when current is turned off) and steady-state measurements (i.e. when cell reaches equilibrium). 

Weppner and Huggins developed GITT in the 1970s for measuring solid state lithium ion diffusion coefficients. They used Li3Sb alloy as a test material, as it is a good approximation of one dimensional lithium ion diffusion into a bulk material, without phase transformations. 

After the cell voltage reached a steady-state value, current was pulsed to determine the effect on measured voltage (i.e. lithiate the electrode by a small amount and then measure how fast it takes the cell to reach equilibrium)



### dQ/dV 

### DST and HPPC for EV applications 

### Three-electrode measurements in R&D 